<a href="https://colab.research.google.com/github/chenowitz/OpenAvalancheProject/blob/sierra_avalanche_center_data/web_scrapper_for_Sierra_Avalanche_Center.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('avalanche.db')
c = conn.cursor()

# Create table for avalanche forecasts
c.execute('''CREATE TABLE IF NOT EXISTS avalanche_forecasts
             (date text, danger_rating text, description text)''')

# URL for Sierra avalanche website
url = 'https://www.sierraavalanchecenter.org/avalanche-forecast'

# Make request to website and parse HTML with BeautifulSoup
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

# Find all past avalanche forecasts on the website
forecasts = soup.find_all('div', {'class': 'forecast'})

# Loop through forecasts and insert into database
for forecast in forecasts:
    date = forecast.find('h4').text.strip()
    danger_rating = forecast.find('h5').text.strip()
    description = forecast.find('div', {'class': 'field-items'}).text.strip()
    c.execute('INSERT INTO avalanche_forecasts VALUES (?, ?, ?)', (date, danger_rating, description))

# Commit changes to database and close connection
conn.commit()
conn.close()


In [12]:
# Database query
conn = sqlite3.connect('avalanche.db')
r_set=conn.execute('''select name from sqlite_master 
		where type = 'table' ''')
for row in r_set:
    print(row)


('avalanche_forecasts',)


In [47]:
from bs4 import SoupStrainer
import re

# 1. Go the the forecast archive prior to Jan 6th
url = 'https://www.sierraavalanchecenter.org/prior-jan-6-2021/archive'

# Source: https://www.appsloveworld.com/php/1947/get-beautifulsoup-to-correctly-parse-php-tags-or-ignore-them
# php_sig = '!!!PHP!!!'
# php_elements = []

# def php_remove(m):
#     php_elements.append(m.group())
#     return php_sig

# def php_add(m):
#     return php_elements.pop(0)

# # Pre-parse HTML to remove all PHP elements
# html = "<html>"
# html = re.sub(r'<\?php.*?\?>', php_remove, html, flags=re.S+re.M)

# # soup = BeautifulSoup(html, "html.parser")

# # Make modifications to the soup
# # Do not remove any elements containing PHP elements

# # Post-parse HTML to replace the PHP elements
# # html = re.sub(php_sig, php_add, soup.prettify())

# # print(html)

# Make request to website and parse HTML with BeautifulSoup
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser',  parse_only=SoupStrainer('td', 'views-field views-field-title active' ))
links = soup.findAll('a')
# https://www.sierraavalanchecenter.org/advisory/2021/jan/8/2021-01-08-065831-avalanche-forecast

individual_forecast_links = []
for link in links:
  href = link.get("href")
  if not ".php" in href:
    individual_forecast_links.append("https://www.sierraavalanchecenter.org"+href)
    # print("https://www.sierraavalanchecenter.org"+href)
# 2. Amass a list of links
for link in individual_forecast_links:
  print(link)

https://www.sierraavalanchecenter.org/advisory/2021/jan/8/2021-01-08-065831-avalanche-forecast
https://www.sierraavalanchecenter.org/advisory/2021/jan/6/2021-01-06-065943-avalanche-forecast
https://www.sierraavalanchecenter.org/advisory/2021/jan/5/2021-01-05-065912-avalanche-forecast
https://www.sierraavalanchecenter.org/advisory/2021/jan/4/2021-01-04-065402-avalanche-forecast
https://www.sierraavalanchecenter.org/advisory/2021/jan/3/2021-01-03-065723-avalanche-forecast
https://www.sierraavalanchecenter.org/advisory/2021/jan/2/2021-01-02-065827-avalanche-forecast
https://www.sierraavalanchecenter.org/advisory/2020/dec/31/2020-12-31-065918-avalanche-forecast
https://www.sierraavalanchecenter.org/advisory/2020/dec/30/2020-12-30-065927-avalanche-forecast
https://www.sierraavalanchecenter.org/advisory/2020/dec/29/2020-12-29-065808-avalanche-forecast
https://www.sierraavalanchecenter.org/advisory/2020/dec/28/2020-12-28-065908-avalanche-forecast
https://www.sierraavalanchecenter.org/advisory

In [54]:
# Scrap for the words avalanche problem:
# def find_avalanche_problem_block(url):
test_urls = ["https://www.sierraavalanchecenter.org/advisory/2021/jan/8/2021-01-08-065831-avalanche-forecast", "https://www.sierraavalanchecenter.org/forecasts/?collapsethemenu=yes#/forecast/1/124600"]
for test_url in test_urls:
  r = requests.get(test_url)
  soup = BeautifulSoup(r.content, 'html.parser')
  links = soup.findAll(string = re.compile('Avalanche Problem'))
  # TODO: Make avalanche problem search case agnostic
  print(links)

['Avalanche Problems Explained', 'Avalanche Problems Explained', 'Avalanche Problem 1: Wind Slab', 'Avalanche Problem 2: Persistent Slab']
['Avalanche Problems Explained', 'Avalanche Problems Explained']


TODO: 
- Spend some time learning about the findAll tags
- Spend some time reading about re and regex